# Cas test debit

In [ ]:
from trustutils import run

run.introduction("G. F.")

run.TRUST_parameters()

### Description: 

Calculs menes en VEF ou VDF.

In [ ]:
from trustutils import run
import os

cas1 = run.addCase(".","test_opt.data")

cas2 = cas1.copy("test_opt2.data")
cas2.substitute("champ_front_uniforme 1 5.","ch_front_input_uniforme { probleme pb initial_value 1 5 nom debit nb_comp 1 }")
run.addCase(cas2)

cas3 = cas1.copy("test_opt3.data")
cas3.substitute("champ_front_uniforme 1 5.","champ_front_fonc_t 1 (1+4*t/10)")
run.addCase(cas3)

cas4 = cas1.copy("champ_normal_var.data")
cas4.substitute("champ_front_debit champ_front_uniforme 1 5."," Boundary_field_inward  { normal_value (1+4*t/10)/0.193136 }")
run.addCase(cas4)

cas5 = run.addCase(".","Champ_front_debit.data")

run.runCases()

cas6 = cas5.copy("testCL.data")

origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY
os.chdir(build_dir)
f_data = open("Champ_front_debit.data","r")
for line in f_data.readlines():
    if line.find("dimension") != -1:
        dim = line.split()[-1]
    if line.find("champ_front_debit") != -1:
        Q_inlet = line.split()[-1]
f_data.close()

f_err = open("Champ_front_debit.err","r")
for line in f_err.readlines():
    if line.find("Area of entree") != -1:
        A_inlet = line.split()[-1]
f_err.close()
os.chdir(origin)

Vit_CLentree = float(Q_inlet)/float(A_inlet)

cas6.substitute("entree frontiere_ouverte_vitesse_imposee champ_front_debit Champ_Front_Uniforme 1  5",
               r"entree frontiere_ouverte_vitesse_imposee Champ_Front_Uniforme %s %.1f 0. " %(dim,Vit_CLentree))

cas6.runCase()

run.addCase(cas6)
run.printCases()


### References
* moi_meme

## VEF test:  calculated flow rate

In [ ]:
from trustutils import plot
import numpy as np

columns=["temps",'debit']
Tableau=plot.Table(columns)  

data=plot.loadText("test_opt_pb_Debit.out",transpose=False)
data[-1][4] = -data[-1][4]
Tableau.addLine([data[-1][0:5:4]],"Calcul")

data=plot.loadText("test_opt2_pb_Debit.out",transpose=False)
data[-1][4] = -data[-1][4]
Tableau.addLine([data[-1][0:5:4]],"Calcul avec champ_input")

data=plot.loadText("test_opt3_pb_Debit.out",transpose=False)
data[-1][4] = -data[-1][4]
Tableau.addLine([data[-1][0:5:4]],"Calcul avec champ_front_fonc_txyz")

data=plot.loadText("champ_normal_var_pb_Debit.out",transpose=False)
data[-1][4] = -data[-1][4]
Tableau.addLine([data[-1][0:5:4]],"Calcul avec Boundary_field_inward variable")

Tableau.addLine([[10,5.]],"voulu") 
display(Tableau)

### Visualization of the velocity field on boundary IN 

The velocity should be uniform even if the surfaces are different.

In [ ]:
from trustutils import visit 

a = visit.Show("./test_opt.lata","Vector","VITESSE_ELEM_IN",mesh="IN")
a.normal3D([-0.0543768,-0.677396,0.733606])
a.up3D([-0.407612, 0.685749,0.602993])
a.plot()

In [ ]:
from trustutils import plot
import numpy as np

Graphique=plot.Graph()
data=plot.loadText("test_opt3_pb_Debit.out")
Graphique.add(data[0],-data[4],marker="-|",color="r",label="Calculation with imposed flow rate")
data=plot.loadText("champ_normal_var_pb_Debit.out")
Graphique.add(data[0],-data[4],marker="-x",color="g",label="Calculation with imposed velocity")
x=np.linspace(0,10,10)
y = 1.+4.*x/10.
Graphique.add(x,y,marker="-",color="b",label="(1+4*x/10)")
Graphique.label("t","debit")


### VDF test:  calculated flow rate

In [ ]:
from trustutils import plot

columns=["temps",'debit']
Tableau=plot.Table(columns)  
data=plot.loadText("Champ_front_debit_pb_Debit.out",transpose=False)
data[2][1] = -data[2][1]
Tableau.addLine([data[2][0:2]],"Calculation with imposed flow rate")
data=plot.loadText("testCL_pb_Debit.out",transpose=False) 
data[2][1] = -data[2][1]
Tableau.addLine([data[2][0:2]],"Calculation with imposed velocity") 
Tableau.addLine([[1.2,5.]],"voulu") 
display(Tableau)

## Data Files

### test_opt

In [ ]:
run.dumpDataset("./test_opt.data") 